# Module 5: Long-Term Memory in LangGraph

**Building on Previous Modules:**
- Module 5.1: Built short-term memory (conversation context)
- Module 5.2: Managed SQLite persistence
- Module 5.3: **Build memory that persists across sessions!**

**What you'll learn:**
- 🧠 Long-term vs Short-term memory
- 💾 LangGraph Store API
- 🗂️ Memory namespaces and organization
- 📊 Semantic memory (facts & profiles)
- 📚 Episodic memory (experiences)
- ⚙️ Procedural memory (rules & prompts)
- 🔍 Memory search and retrieval
- 🎯 Production-ready patterns

**Real HR Use Case:**
Build an HR assistant that:
- Remembers employee preferences across sessions
- Learns from past interactions
- Maintains user profiles
- Adapts its behavior over time

**Time:** 3-4 hours

**Updates for LangChain 1.0:**
- ✅ Fixed: `prompt=` → `system_prompt=`
- ✅ Fixed: Using `ToolRuntime` instead of `InjectedStore`
- ✅ Fixed: `SqliteSaver` usage pattern
- ✅ Fixed: Context-based tool access

## Setup: Install Dependencies

In [20]:
# Install LangChain 1.0 and required packages
!pip install --upgrade pip
!pip install --pre -U langchain langchain-openai langgraph langchain-community
!pip install -U langgraph-checkpoint-sqlite  # For SQLite persistence

# Check versions
import langchain
import langgraph
print(f"LangChain version: {langchain.__version__}")
print(f"LangGraph version: {langgraph.__version__}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
LangChain version: 1.0.0


AttributeError: module 'langgraph' has no attribute '__version__'

## Setup: Configure API Keys & Imports

In [21]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Common imports
from dataclasses import dataclass
from typing_extensions import TypedDict
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain.tools import tool, ToolRuntime
from langgraph.store.memory import InMemoryStore
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import SystemMessage
import json
from datetime import datetime
import uuid
import sqlite3

print("✅ Setup complete!")

✅ Setup complete!


---
# Part 1: Understanding Long-Term Memory 🧠

## Short-Term vs Long-Term Memory

### Short-Term Memory (Previous Module)
```
Session 1:
User: "My name is Priya"
Agent: "Hello Priya!"

[App restarts]

Session 2 (Same thread):
User: "What's my name?"
Agent: "Your name is Priya!" ✅
```

### Long-Term Memory (This Module)
```
Session 1:
User: "My name is Priya and I prefer morning shifts"
Agent: "Got it! I'll remember your preference."

[Days later, different thread]

Session 2 (Different thread):
User: "Schedule a meeting for me"
Agent: "I'll schedule it in the morning as you prefer!" ✅
```

## Key Differences

| Feature | Short-Term | Long-Term |
|---------|-----------|----------|
| **Scope** | Single thread/session | Across all threads |
| **Storage** | Checkpoints | Store (Database) |
| **Lifespan** | Session duration | Permanent |
| **Use Case** | "What did I just say?" | "What are my preferences?" |
| **Key** | thread_id | user_id/org_id |

## Memory Types

### 1. **Semantic Memory** - Facts & Profiles
- User preferences, settings
- Company policies, employee data
- Example: "Priya prefers email notifications"

### 2. **Episodic Memory** - Experiences
- Past conversations and interactions
- Historical decisions and outcomes
- Example: "Last time we solved this by..."

### 3. **Procedural Memory** - Rules & Behavior
- Custom instructions per user
- Learned workflows
- Example: "Always be formal with executives"

---
# Part 2: LangGraph Store API 💾

**Key Concept:** Store is a key-value database for long-term memories.

**Core Operations:**
- `put()` - Save/Update memory
- `get()` - Retrieve memory
- `search()` - Find memories
- `delete()` - Remove memory

**Organization:**
- **Namespace**: Group related memories (like folders)
- **Key**: Unique identifier within namespace (like filename)

## Lab 1.1: Basic Store Operations

In [22]:
# Create an in-memory store
store = InMemoryStore()

print("=" * 70)
print("Lab 1.1: Basic Store Operations")
print("=" * 70 + "\n")

# 1. PUT - Save employee profile
employee_profile = {
    "name": "Priya Sharma",
    "employee_id": "101",
    "department": "Engineering",
    "preferences": {
        "notification_method": "email",
        "work_hours": "9 AM - 5 PM",
        "preferred_meeting_time": "morning"
    },
    "leave_balance": 12
}

# Save to store
# Namespace: "employees" (like a folder)
# Key: "user_101" (like a filename)
store.put(
    namespace=("employees",),  # Tuple for hierarchical namespaces
    key="user_101",
    value=employee_profile
)
print("✅ Saved employee profile for Priya")

# 2. GET - Retrieve employee profile
retrieved = store.get(
    namespace=("employees",),
    key="user_101"
)
print(f"\n📥 Retrieved profile:")
print(f"Name: {retrieved.value['name']}")
print(f"Department: {retrieved.value['department']}")
print(f"Preferred meeting time: {retrieved.value['preferences']['preferred_meeting_time']}")

# 3. UPDATE - Modify profile
employee_profile["leave_balance"] = 10  # Used 2 days
store.put(
    namespace=("employees",),
    key="user_101",
    value=employee_profile
)
print("\n✅ Updated leave balance")

# 4. Multiple employees
store.put(
    namespace=("employees",),
    key="user_102",
    value={
        "name": "Rahul Verma",
        "employee_id": "102",
        "department": "Marketing",
        "leave_balance": 8
    }
)
print("✅ Saved second employee profile")

# 5. SEARCH - Find all employees
all_employees = store.search(("employees",))
print(f"\n📊 Total employees in store: {len(all_employees)}")
for emp in all_employees:
    print(f"  - {emp.value['name']} ({emp.value['department']})")

print("\n✅ Store operations complete!")

Lab 1.1: Basic Store Operations

✅ Saved employee profile for Priya

📥 Retrieved profile:
Name: Priya Sharma
Department: Engineering
Preferred meeting time: morning

✅ Updated leave balance
✅ Saved second employee profile

📊 Total employees in store: 2
  - Priya Sharma (Engineering)
  - Rahul Verma (Marketing)

✅ Store operations complete!


## Lab 1.2: Hierarchical Namespaces

**Use Case:** Organize memories by company → department → user

In [23]:
store = InMemoryStore()

print("=" * 70)
print("Lab 1.2: Hierarchical Namespaces")
print("=" * 70 + "\n")

# Hierarchical structure: company → department → user
store.put(
    namespace=("acme_corp", "engineering", "user_101"),
    key="profile",
    value={"name": "Priya Sharma", "role": "Senior Developer"}
)

store.put(
    namespace=("acme_corp", "marketing", "user_102"),
    key="profile",
    value={"name": "Rahul Verma", "role": "Marketing Manager"}
)

store.put(
    namespace=("acme_corp", "engineering", "user_103"),
    key="profile",
    value={"name": "Anjali Patel", "role": "Tech Lead"}
)

print("✅ Saved 3 employee profiles with hierarchical namespaces\n")

# Search by department
print("🔍 Searching Engineering department:")
eng_employees = store.search(("acme_corp", "engineering"))
for emp in eng_employees:
    print(f"  - {emp.value['name']}: {emp.value['role']}")

print("\n🔍 Searching Marketing department:")
marketing_employees = store.search(("acme_corp", "marketing"))
for emp in marketing_employees:
    print(f"  - {emp.value['name']}: {emp.value['role']}")

print("\n🔍 Searching entire company:")
all_employees = store.search(("acme_corp",))
print(f"Total employees: {len(all_employees)}")

print("\n✅ Hierarchical organization makes data management easy!")

Lab 1.2: Hierarchical Namespaces

✅ Saved 3 employee profiles with hierarchical namespaces

🔍 Searching Engineering department:
  - Priya Sharma: Senior Developer
  - Anjali Patel: Tech Lead

🔍 Searching Marketing department:
  - Rahul Verma: Marketing Manager

🔍 Searching entire company:
Total employees: 3

✅ Hierarchical organization makes data management easy!


---
# Part 3: Semantic Memory - User Profiles 📊

**Use Case:** Remember facts about users across sessions

## Lab 2.1: Reading Long-Term Memory in Tools

In [26]:
# Initialize store and pre-populate with employee data
store = InMemoryStore()

# Pre-populate store with employee profiles
store.put(
    namespace=("users",),
    key="user_101",
    value={
        "name": "Priya Sharma",
        "employee_id": "101",
        "department": "Engineering",
        "preferences": {
            "notification_method": "email",
            "meeting_time": "morning"
        },
        "leave_balance": 12
    }
)

# Define Context for user identification
@dataclass
class Context:
    user_id: str

# Tool that reads from long-term memory using ToolRuntime
@tool
def get_user_preferences(runtime: ToolRuntime) -> str:
    """Get user preferences from long-term memory."""
    # Access store and context from runtime
    store = runtime.store
    user_id = runtime.context.user_id

    # Get user profile from store
    user_profile = store.get(namespace=("users",), key=user_id)

    if user_profile is None:
        return f"No profile found for {user_id}"

    prefs = user_profile.value.get("preferences", {})
    name = user_profile.value.get("name", "Unknown")

    return f"""{name}'s preferences:
- Notification method: {prefs.get('notification_method', 'Not set')}
- Preferred meeting time: {prefs.get('meeting_time', 'Not set')}
- Leave balance: {user_profile.value.get('leave_balance', 0)} days
"""

# Create agent with store
# Create SQLite connection for checkpointer
conn = sqlite3.connect(":memory:", check_same_thread=False)
checkpointer = SqliteSaver(conn)

agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_user_preferences],
    checkpointer=checkpointer,
    store=store,
    context_schema=Context,
    system_prompt="You are an HR assistant with access to employee profiles."
)

print("=" * 70)
print("Lab 2.1: Reading Long-Term Memory")
print("=" * 70 + "\n")

# Test 1: Get preferences
result = agent.invoke(
    {"messages": "What are my preferences?"},
    config={"configurable": {"thread_id": "session_1"}},
    context=Context(user_id="user_101")
)
print("Test 1 - Get preferences:")
print(result['messages'][-1].content)

# Test 2: Different session, same user - memory persists!
result = agent.invoke(
    {"messages": "When should I schedule meetings?"},
    config={"configurable": {"thread_id": "session_2"}},  # Different thread!
    context=Context(user_id="user_101")
)
print("\nTest 2 - Different session (memory persists):")
print(result['messages'][-1].content)

print("\n✅ Long-term memory works across different sessions!")

Lab 2.1: Reading Long-Term Memory

Test 1 - Get preferences:
Your preferences are as follows:

- **Notification method:** Email
- **Preferred meeting time:** Morning
- **Leave balance:** 12 days

Test 2 - Different session (memory persists):
Priya Sharma prefers to schedule meetings in the morning. You may want to consider this preference when planning upcoming meetings.

✅ Long-term memory works across different sessions!


## Lab 2.2: Writing Long-Term Memory from Tools

In [28]:
store = InMemoryStore()

@dataclass
class Context:
    user_id: str

# Schema for user preferences
class UserPreferences(TypedDict):
    notification_method: str  # 'email', 'slack', 'phone'
    meeting_time: str  # 'morning', 'afternoon', 'evening'
    work_style: str  # 'collaborative', 'independent', 'hybrid'

@tool
def save_user_preferences(
    preferences: UserPreferences,
    runtime: ToolRuntime
) -> str:
    """Save or update user preferences in long-term memory."""
    store = runtime.store
    user_id = runtime.context.user_id

    # Get existing profile or create new
    existing = store.get(namespace=("users",), key=user_id)

    if existing:
        profile = existing.value
        profile["preferences"] = preferences
    else:
        profile = {
            "user_id": user_id,
            "preferences": preferences
        }

    # Save to store
    store.put(
        namespace=("users",),
        key=user_id,
        value=profile
    )

    return f"✅ Saved preferences for {user_id}: {preferences}"

@tool
def get_user_info(runtime: ToolRuntime) -> str:
    """Get user information from long-term memory."""
    store = runtime.store
    user_id = runtime.context.user_id

    profile = store.get(namespace=("users",), key=user_id)

    if profile is None:
        return f"No profile found for {user_id}"

    return f"Profile for {user_id}: {profile.value}"

# Create agent with write capabilities
conn = sqlite3.connect(":memory:", check_same_thread=False)
checkpointer = SqliteSaver(conn)

agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[save_user_preferences, get_user_info],
    checkpointer=checkpointer,
    store=store,
    context_schema=Context,
    system_prompt="""You are an HR assistant that helps users manage their preferences.
    When users tell you their preferences, save them using the save_user_preferences tool."""
)

print("=" * 70)
print("Lab 2.2: Writing Long-Term Memory")
print("=" * 70 + "\n")

# User sets preferences
result = agent.invoke({
    "messages": """Hi! I prefer:
    - Email notifications
    - Morning meetings
    - Independent work style
    Please save these preferences."""
},
    config={"configurable": {"thread_id": "onboarding_session"}},
    context=Context(user_id="user_101")
)
print("User sets preferences:")
print(result['messages'][-1].content)

# Different session - verify memory persists
result = agent.invoke(
    {"messages": "What are my preferences?"},
    config={"configurable": {"thread_id": "different_session"}},
    context=Context(user_id="user_101")
)
print("\nDifferent session - retrieve preferences:")
print(result['messages'][-1].content)

print("\n✅ Preferences saved and retrieved across sessions!")

Lab 2.2: Writing Long-Term Memory

User sets preferences:
Your preferences have been successfully saved:
- Notification Method: Email notifications
- Meeting Time: Morning meetings
- Work Style: Independent work style

If you need anything else, feel free to ask!

Different session - retrieve preferences:
Your preferences are as follows:

- **Notification Method:** Email notifications
- **Meeting Time:** Morning meetings
- **Work Style:** Independent work style

✅ Preferences saved and retrieved across sessions!


---
# Part 4: Episodic Memory - Learning from Experience 📚

**Use Case:** Remember past interactions to improve responses

## Lab 3.1: Storing Interaction History

In [30]:
store = InMemoryStore()

@dataclass
class Context:
    user_id: str

@tool
def save_interaction(
    interaction_type: str,
    description: str,
    resolution: str,
    runtime: ToolRuntime
) -> str:
    """Save an interaction to episodic memory. Type should be 'question', 'issue', or 'feedback'."""
    store = runtime.store
    user_id = runtime.context.user_id

    interaction_id = str(uuid.uuid4())[:8]

    interaction_data = {
        "interaction_id": interaction_id,
        "type": interaction_type,
        "description": description,
        "resolution": resolution,
        "timestamp": datetime.now().isoformat()
    }

    # Save interaction
    store.put(
        namespace=("interactions", user_id),
        key=interaction_id,
        value=interaction_data
    )

    return f"✅ Saved interaction {interaction_id}"

@tool
def get_past_interactions(runtime: ToolRuntime) -> str:
    """Retrieve past interactions for the current user."""
    store = runtime.store
    user_id = runtime.context.user_id

    interactions = store.search(namespace=("interactions", user_id))

    if not interactions:
        return f"No past interactions found for {user_id}"

    result = f"Past interactions for {user_id}:\n\n"
    for item in interactions:
        data = item.value
        result += f"Type: {data['type']}\n"
        result += f"Description: {data['description']}\n"
        result += f"Resolution: {data['resolution']}\n"
        result += f"Date: {data['timestamp'][:10]}\n\n"

    return result

conn = sqlite3.connect(":memory:", check_same_thread=False)
checkpointer = SqliteSaver(conn)

agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[save_interaction, get_past_interactions],
    checkpointer=checkpointer,
    store=store,
    context_schema=Context,
    system_prompt="""You are an HR assistant that learns from past interactions.
    Save important interactions so you can reference them later."""
)

print("=" * 70)
print("Lab 3.1: Episodic Memory")
print("=" * 70 + "\n")

# First interaction - report issue
result = agent.invoke({
    "messages": """I had an issue accessing the leave portal.
    It kept showing 'Access Denied'. We fixed it by resetting my credentials."""
},
    config={"configurable": {"thread_id": "issue_session_1"}},
    context=Context(user_id="user_101")
)
print("Session 1 - Report issue:")
print(result['messages'][-1].content)

# Different session - similar issue
result = agent.invoke({
    "messages": """I'm having trouble accessing the leave portal again.
    What should I do?"""
},
    config={"configurable": {"thread_id": "issue_session_2"}},
    context=Context(user_id="user_101")
)
print("\nSession 2 - Similar issue (learns from past):")
print(result['messages'][-1].content)

print("\n✅ Agent learned from past experience!")

Lab 3.1: Episodic Memory

Session 1 - Report issue:
I've saved the interaction regarding the issue you faced accessing the leave portal and its resolution. If you need to refer back to it later, just let me know!

Session 2 - Similar issue (learns from past):
It seems you are having trouble accessing the leave portal. You should check your login credentials and consider resetting your password. If the issue persists, I recommend contacting IT support for further assistance.

✅ Agent learned from past experience!


---
# Part 5: Procedural Memory - Adaptive Behavior ⚙️

**Use Case:** Learn and adapt system behavior over time

## Lab 4.1: User-Specific Instructions

In [34]:
store = InMemoryStore()

@dataclass
class Context:
    user_id: str

# Pre-populate with user-specific instructions
store.put(
    namespace=("instructions",),
    key="user_101",
    value={
        "communication_style": "professional and concise",
        "special_instructions": [
            "Always CC manager on leave requests",
            "Prefer email over chat",
            "Send reminders 2 days before deadlines"
        ]
    }
)

store.put(
    namespace=("instructions",),
    key="user_102",
    value={
        "communication_style": "friendly and detailed",
        "special_instructions": [
            "Provide step-by-step guides",
            "Use examples when explaining"
        ]
    }
)

@tool
def get_user_instructions(runtime: ToolRuntime) -> str:
    """Get customized instructions for interacting with the current user."""
    store = runtime.store
    user_id = runtime.context.user_id

    instructions = store.get(namespace=("instructions",), key=user_id)

    if instructions is None:
        return "No special instructions for this user."

    data = instructions.value
    result = f"Instructions for {user_id}:\n"
    result += f"Communication style: {data['communication_style']}\n"
    result += "Special instructions:\n"
    for instr in data['special_instructions']:
        result += f"  - {instr}\n"

    return result

conn = sqlite3.connect(":memory:", check_same_thread=False)
checkpointer = SqliteSaver(conn)

agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_user_instructions],
    checkpointer=checkpointer,
    store=store,
    context_schema=Context,
    system_prompt="""You are an adaptive HR assistant.
    Before responding, check user-specific instructions and adapt your behavior."""
)

print("=" * 70)
print("Lab 4.1: User-Specific Instructions")
print("=" * 70 + "\n")

# Test with user_101 (professional style)
result = agent.invoke(
    {"messages": "How do I apply for leave?"},
    config={"configurable": {"thread_id": "user_101_session"}},
    context=Context(user_id="user_101")
)
print("User 101 (professional style):")
print(result['messages'][-1].content)

# Test with user_102 (friendly, detailed style)
result = agent.invoke(
    {"messages": "How do I apply for leave?"},
    config={"configurable": {"thread_id": "user_102_session"}},
    context=Context(user_id="user_102")
)
print("\nUser 102 (friendly, detailed style):")
print(result['messages'][-1].content)

print("\n✅ Agent adapted behavior based on user!")

Lab 4.1: User-Specific Instructions

User 101 (professional style):
To apply for leave, please follow these steps:

1. Draft an email addressed to the HR department.
2. In the CC field, include your manager.
3. Clearly state your leave request in the subject line and provide details in the body of the email (dates, reason for leave, etc.).
4. Send the email.

If needed, remember to send a reminder email 2 days before any relevant deadlines.

User 102 (friendly, detailed style):
Applying for leave typically involves several steps, and the exact process can vary by organization. Here’s a friendly and detailed step-by-step guide to help you through:

### Step 1: Check Your Company Policy
Before you proceed with your application, it’s important to understand your company’s leave policy. Look for the following:
- Types of leave available (sick leave, vacation, personal leave, etc.)
- Required notice period
- Application procedure (e.g., online portal, paper form)

**Example:** If your compa

---
# Part 6: Production Pattern - Complete HR Assistant 🎯

**Combining all memory types for production use**

## Lab 5.1: Complete Production HR Assistant

In [36]:
# Production HR Assistant with all memory types
store = InMemoryStore()

@dataclass
class Context:
    user_id: str

# Define comprehensive tool suite
@tool
def get_employee_profile(runtime: ToolRuntime) -> str:
    """Get employee profile (Semantic Memory)."""
    store = runtime.store
    user_id = runtime.context.user_id

    profile = store.get(namespace=("profiles",), key=user_id)
    if profile:
        return json.dumps(profile.value, indent=2)
    return f"No profile found for {user_id}"

@tool
def save_employee_info(
    info: dict,
    runtime: ToolRuntime
) -> str:
    """Save employee information."""
    store = runtime.store
    user_id = runtime.context.user_id

    store.put(namespace=("profiles",), key=user_id, value=info)
    return f"✅ Saved info for {user_id}"

@tool
def log_interaction(
    interaction_summary: str,
    runtime: ToolRuntime
) -> str:
    """Log interaction (Episodic Memory)."""
    store = runtime.store
    user_id = runtime.context.user_id

    interaction_id = str(uuid.uuid4())[:8]
    store.put(
        namespace=("interactions", user_id),
        key=interaction_id,
        value={
            "summary": interaction_summary,
            "timestamp": datetime.now().isoformat()
        }
    )
    return f"✅ Logged interaction {interaction_id}"

@tool
def get_user_instructions(runtime: ToolRuntime) -> str:
    """Get user-specific instructions (Procedural Memory)."""
    store = runtime.store
    user_id = runtime.context.user_id

    instructions = store.get(namespace=("instructions",), key=user_id)
    if instructions:
        return json.dumps(instructions.value, indent=2)
    return "No special instructions"

# Create production agent
conn = sqlite3.connect(":memory:", check_same_thread=False)
checkpointer = SqliteSaver(conn)

production_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[
        get_employee_profile,
        save_employee_info,
        log_interaction,
        get_user_instructions
    ],
    checkpointer=checkpointer,
    store=store,
    context_schema=Context,
    system_prompt="""You are a production HR assistant with:

    - Semantic Memory: Employee profiles and preferences
    - Episodic Memory: Past interactions and resolutions
    - Procedural Memory: User-specific behavioral instructions

    Before responding:
    1. Check user profile for context
    2. Check user instructions for preferences
    3. Log important interactions

    Be helpful, professional, and personalized."""
)

print("=" * 70)
print("Lab 5.1: Production HR Assistant")
print("=" * 70 + "\n")

# Simulate multi-session usage
# Session 1: Onboarding
result = production_agent.invoke(
    {"messages": "Hi! I'm new employee. I'm Priya from Engineering."},
    config={"configurable": {"thread_id": "onboarding_201"}},
    context=Context(user_id="user_201")
)
print("Session 1 (Onboarding):")
print(result['messages'][-1].content)

# Session 2: Different day, different thread
result = production_agent.invoke(
    {"messages": "How do I apply for leave?"},
    config={"configurable": {"thread_id": "leave_query_201"}},
    context=Context(user_id="user_201")
)
print("\nSession 2 (Different day - remembers user):")
print(result['messages'][-1].content)

print("\n✅ Production agent with complete long-term memory!")

Lab 5.1: Production HR Assistant

Session 1 (Onboarding):
Welcome to the team, Priya! It's great to have you in the Engineering department. Since you're new, I can assist you with any questions you may have about the company, your role, or any other topics. Please let me know how I can help you!

Session 2 (Different day - remembers user):
It seems I couldn't find your employee profile or any specific preferences regarding leave applications. However, I can guide you through a general process:

1. **Check the Leave Policy**: Review your company's leave policy to understand the types of leave available (e.g., sick leave, vacation leave) and any requirements for application.

2. **Submit a Request**: Typically, you'd need to fill out a leave application form. This could be done through your HR management system if your company uses one. If not, you might have to write a formal email.

3. **Provide Details**: Include your name, the dates you wish to take off, the reason for your leave, an

---
# Summary & Best Practices

## Long-Term Memory Comparison

| Memory Type | What to Store | When to Use | Example |
|-------------|--------------|-------------|----------|
| **Semantic** | Facts, profiles | User preferences | "Prefers email" |
| **Episodic** | Past interactions | Learning from experience | "Fixed by resetting" |
| **Procedural** | Behavior rules | Personalization | "Be formal with execs" |

## Key Patterns

### Using ToolRuntime (LangChain 1.0)
```python
@dataclass
class Context:
    user_id: str

@tool
def my_tool(runtime: ToolRuntime[Context]) -> str:
    store = runtime.store  # Access store
    user_id = runtime.context.user_id  # Access context
    # Use store operations
    return "result"

agent = create_agent(
    model="...",
    tools=[my_tool],
    store=store,
    context_schema=Context
)

# Invoke with context
agent.invoke(
    {"messages": "..."},
    context=Context(user_id="user_123")
)
```

## Production Checklist

✅ **Use persistent store** (not InMemoryStore in production)  
✅ **Organize with namespaces** (user_id, org_id, dept)  
✅ **Combine with short-term memory** (checkpointer + store)  
✅ **Use ToolRuntime pattern** for LangChain 1.0  
✅ **Pass context on invoke** with user_id  
✅ **Handle missing data** gracefully  

## LangChain 1.0 Updates Applied

✅ Changed all `prompt=` to `system_prompt=`  
✅ Using `ToolRuntime[Context]` instead of `InjectedStore()`  
✅ Fixed SqliteSaver usage with manual connection  
✅ Added `context_schema` and `context` parameters  

---

**Remember:** Long-term memory makes agents truly useful!